In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Yearly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 23000    17

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9903832,1.535759e-07,5.174030,-0.94405361,0.47491778,0.6782989,0.3751645,0.9361374,3.2285135,0.39993885,0.28707837,-0.1820491,0.22316867,31
2,1,0,1,0.7455956,1.919834e-04,-2.797630,2.13419228,0.27486845,0.4580795,0.8010614,0.7084840,0.8208482,0.12753274,0.22214291,-0.3185895,0.37163112,31
3,1,0,1,0.9993711,1.038094e-09,5.497375,-0.01972653,0.20035534,0.4491332,0.3055037,0.8966488,2.9734257,-0.17814744,0.37277176,-0.6280637,1.00110959,31
4,1,0,1,0.9991043,5.484677e-09,5.514672,-0.12400209,-0.18881555,0.3260450,0.3087948,0.8904246,2.8852573,-0.45891796,0.60342537,-0.7758825,1.24511944,31
5,1,0,1,0.9941526,1.000137e-07,5.421624,0.72042474,0.15921568,0.4025341,0.3435267,0.8685223,2.4451391,0.02494266,0.16237458,-0.5492186,0.61790122,31
6,1,0,1,0.9642112,2.683180e-05,4.078327,0.73445937,0.02824047,0.1412002,0.5294464,0.7107351,1.3317587,0.22505892,0.08313085,0.1296658,0.09734006,19


In [6]:
dlist= load('year_thetaf_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=10

In [7]:
dim(MASE)

[1] 23000     5     4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
    0     1     2     3     4 
 2930  1622  2158  3732 12558 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

0
2
3
1
4
4
4
2
4
4
4


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9903832,1.535759e-07,5.174030,-0.94405361,0.47491778,0.6782989,0.3751645,0.9361374,3.2285135,0.39993885,0.28707837,-0.1820491,0.22316867,31
2,1,0,1,0.7455956,1.919834e-04,-2.797630,2.13419228,0.27486845,0.4580795,0.8010614,0.7084840,0.8208482,0.12753274,0.22214291,-0.3185895,0.37163112,31
3,1,0,1,0.9993711,1.038094e-09,5.497375,-0.01972653,0.20035534,0.4491332,0.3055037,0.8966488,2.9734257,-0.17814744,0.37277176,-0.6280637,1.00110959,31
4,1,0,1,0.9991043,5.484677e-09,5.514672,-0.12400209,-0.18881555,0.3260450,0.3087948,0.8904246,2.8852573,-0.45891796,0.60342537,-0.7758825,1.24511944,31
5,1,0,1,0.9941526,1.000137e-07,5.421624,0.72042474,0.15921568,0.4025341,0.3435267,0.8685223,2.4451391,0.02494266,0.16237458,-0.5492186,0.61790122,31
6,1,0,1,0.9642112,2.683180e-05,4.078327,0.73445937,0.02824047,0.1412002,0.5294464,0.7107351,1.3317587,0.22505892,0.08313085,0.1296658,0.09734006,19


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,0,1,0.9903832,1.535759e-07,5.174030,-0.94405361,0.47491778,0.6782989,0.3751645,0.9361374,3.2285135,0.39993885,0.28707837,-0.1820491,0.22316867,31
2,1,0,1,0.7455956,1.919834e-04,-2.797630,2.13419228,0.27486845,0.4580795,0.8010614,0.7084840,0.8208482,0.12753274,0.22214291,-0.3185895,0.37163112,31
3,1,0,1,0.9993711,1.038094e-09,5.497375,-0.01972653,0.20035534,0.4491332,0.3055037,0.8966488,2.9734257,-0.17814744,0.37277176,-0.6280637,1.00110959,31
4,1,0,1,0.9991043,5.484677e-09,5.514672,-0.12400209,-0.18881555,0.3260450,0.3087948,0.8904246,2.8852573,-0.45891796,0.60342537,-0.7758825,1.24511944,31
5,1,0,1,0.9941526,1.000137e-07,5.421624,0.72042474,0.15921568,0.4025341,0.3435267,0.8685223,2.4451391,0.02494266,0.16237458,-0.5492186,0.61790122,31
6,1,0,1,0.9642112,2.683180e-05,4.078327,0.73445937,0.02824047,0.1412002,0.5294464,0.7107351,1.3317587,0.22505892,0.08313085,0.1296658,0.09734006,19


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:2.158514 
[2]	train-rmse:1.742256 
[3]	train-rmse:1.489099 
[4]	train-rmse:1.340889 
[5]	train-rmse:1.258073 
[6]	train-rmse:1.212298 
[7]	train-rmse:1.182897 
[8]	train-rmse:1.163587 
[9]	train-rmse:1.149362 
[10]	train-rmse:1.140159 
[11]	train-rmse:1.133881 
[12]	train-rmse:1.131376 
[13]	train-rmse:1.126160 
[14]	train-rmse:1.122253 
[15]	train-rmse:1.116136 
[16]	train-rmse:1.110108 
[17]	train-rmse:1.106567 
[18]	train-rmse:1.104218 
[19]	train-rmse:1.097043 
[20]	train-rmse:1.092242 
[21]	train-rmse:1.087303 
[22]	train-rmse:1.084730 
[23]	train-rmse:1.075470 
[24]	train-rmse:1.072612 
[25]	train-rmse:1.066474 
[26]	train-rmse:1.062442 
[27]	train-rmse:1.058431 
[28]	train-rmse:1.053957 
[29]	train-rmse:1.049472 
[30]	train-rmse:1.047845 
[31]	train-rmse:1.044107 
[32]	train-rmse:1.042712 
[33]	train-rmse:1.035275 
[34]	train-rmse:1.031096 
[35]	train-rmse:1.029948 
[36]	train-rmse:1.028160 
[37]	train-rmse:1.026214 
[38]	train-rmse:1.023318 
[39]	train-rmse:1.021

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.397855 
[2]	train-mlogloss:1.277738 
[3]	train-mlogloss:1.196971 
[4]	train-mlogloss:1.138458 
[5]	train-mlogloss:1.094312 
[6]	train-mlogloss:1.059847 
[7]	train-mlogloss:1.031781 
[8]	train-mlogloss:1.009685 
[9]	train-mlogloss:0.988899 
[10]	train-mlogloss:0.970740 
[11]	train-mlogloss:0.956190 
[12]	train-mlogloss:0.940102 
[13]	train-mlogloss:0.927342 
[14]	train-mlogloss:0.916759 
[15]	train-mlogloss:0.908690 
[16]	train-mlogloss:0.897425 
[17]	train-mlogloss:0.889513 
[18]	train-mlogloss:0.881631 
[19]	train-mlogloss:0.872705 
[20]	train-mlogloss:0.865144 
[21]	train-mlogloss:0.860449 
[22]	train-mlogloss:0.853472 
[23]	train-mlogloss:0.847163 
[24]	train-mlogloss:0.840449 
[25]	train-mlogloss:0.834729 
[26]	train-mlogloss:0.828786 
[27]	train-mlogloss:0.821251 
[28]	train-mlogloss:0.816515 
[29]	train-mlogloss:0.808764 
[30]	train-mlogloss:0.800555 
[31]	train-mlogloss:0.795000 
[32]	train-mlogloss:0.790398 
[33]	train-mlogloss:0.784302 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3402
[LightGBM] [Info] Number of data points in the train set: 16214, number of used features: 14
[LightGBM] [Info] Start training from score 2.924941


In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007138 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3402
[LightGBM] [Info] Number of data points in the train set: 16214, number of used features: 14
[LightGBM] [Info] Start training from score -2.062684
[LightGBM] [Info] Start training from score -2.645244
[LightGBM] [Info] Start training from score -2.351499
[LightGBM] [Info] Start training from score -1.812826
[LightGBM] [Info] Start training from score -0.609761


In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [29]:
start_time = Sys.time()

In [30]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [31]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [32]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:2.031002 
[2]	train-rmse:1.669973 
[3]	train-rmse:1.457106 
[4]	train-rmse:1.331721 
[5]	train-rmse:1.261543 
[6]	train-rmse:1.216325 
[7]	train-rmse:1.194168 
[8]	train-rmse:1.175312 
[9]	train-rmse:1.164818 
[10]	train-rmse:1.153477 
[11]	train-rmse:1.144681 
[12]	train-rmse:1.137779 
[13]	train-rmse:1.130611 
[14]	train-rmse:1.125671 
[15]	train-rmse:1.119407 
[16]	train-rmse:1.113970 
[17]	train-rmse:1.110961 
[18]	train-rmse:1.101511 
[19]	train-rmse:1.094694 
[20]	train-rmse:1.091653 
[21]	train-rmse:1.085903 
[22]	train-rmse:1.084576 
[23]	train-rmse:1.081267 
[24]	train-rmse:1.074729 
[25]	train-rmse:1.072663 
[26]	train-rmse:1.067405 
[27]	train-rmse:1.064180 
[28]	train-rmse:1.057560 
[29]	train-rmse:1.053211 
[30]	train-rmse:1.049924 
[31]	train-rmse:1.046348 
[32]	train-rmse:1.043835 
[33]	train-rmse:1.040003 
[34]	train-rmse:1.037268 
[35]	train-rmse:1.035954 
[36]	train-rmse:1.034786 
[37]	train-rmse:1.033670 
[38]	train-rmse:1.030586 
[39]	train-rmse:1.026

In [33]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.443403 
[2]	train-mlogloss:1.343605 
[3]	train-mlogloss:1.272997 
[4]	train-mlogloss:1.221468 
[5]	train-mlogloss:1.178781 
[6]	train-mlogloss:1.144452 
[7]	train-mlogloss:1.115995 
[8]	train-mlogloss:1.091559 
[9]	train-mlogloss:1.070846 
[10]	train-mlogloss:1.052189 
[11]	train-mlogloss:1.036957 
[12]	train-mlogloss:1.023914 
[13]	train-mlogloss:1.011506 
[14]	train-mlogloss:0.999135 
[15]	train-mlogloss:0.989785 
[16]	train-mlogloss:0.979946 
[17]	train-mlogloss:0.971828 
[18]	train-mlogloss:0.960725 
[19]	train-mlogloss:0.953850 
[20]	train-mlogloss:0.948391 
[21]	train-mlogloss:0.942561 
[22]	train-mlogloss:0.933557 
[23]	train-mlogloss:0.926806 
[24]	train-mlogloss:0.921163 
[25]	train-mlogloss:0.913841 
[26]	train-mlogloss:0.908058 
[27]	train-mlogloss:0.901028 
[28]	train-mlogloss:0.893359 
[29]	train-mlogloss:0.888810 
[30]	train-mlogloss:0.883349 
[31]	train-mlogloss:0.876972 
[32]	train-mlogloss:0.872125 
[33]	train-mlogloss:0.865660 
[34]	train-mlogloss

In [34]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013295 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3402
[LightGBM] [Info] Number of data points in the train set: 16214, number of used features: 14
[LightGBM] [Info] Start training from score 2.634575


In [35]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3402
[LightGBM] [Info] Number of data points in the train set: 16214, number of used features: 14
[LightGBM] [Info] Start training from score -1.827291
[LightGBM] [Info] Start training from score -2.355392
[LightGBM] [Info] Start training from score -2.126285
[LightGBM] [Info] Start training from score -1.614942
[LightGBM] [Info] Start training from score -0.853050


In [36]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [63]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,entropy,⋯,diff1_acf10,diff2_acf1,diff2_acf10,length,user_time,system_time,elapsed_time,user_time.1,system_time.1,elapsed_time.1
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,1,0.9903832,1.535759e-07,5.174030,-0.94405361,0.47491778,0.6782989,0.3751645,⋯,0.28707837,-0.1820491,0.22316867,31,0.01651573,0.01651573,0.01651573,0,0,0
2,1,0,1,0.7455956,1.919834e-04,-2.797630,2.13419228,0.27486845,0.4580795,0.8010614,⋯,0.22214291,-0.3185895,0.37163112,31,0.01546788,0.01546788,0.01546788,0,0,0
3,1,0,1,0.9993711,1.038094e-09,5.497375,-0.01972653,0.20035534,0.4491332,0.3055037,⋯,0.37277176,-0.6280637,1.00110959,31,0.01541162,0.01541162,0.01541162,0,0,0
4,1,0,1,0.9991043,5.484677e-09,5.514672,-0.12400209,-0.18881555,0.3260450,0.3087948,⋯,0.60342537,-0.7758825,1.24511944,31,0.01328015,0.01328015,0.01328015,0,0,0
5,1,0,1,0.9941526,1.000137e-07,5.421624,0.72042474,0.15921568,0.4025341,0.3435267,⋯,0.16237458,-0.5492186,0.61790122,31,0.01211190,0.01211190,0.01211190,0,0,0
6,1,0,1,0.9642112,2.683180e-05,4.078327,0.73445937,0.02824047,0.1412002,0.5294464,⋯,0.08313085,0.1296658,0.09734006,19,0.01194334,0.01194334,0.01194334,0,0,0


In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  1.171583295  2.156448126  3.519929886  2.976366997  3.802012682
    [6]  3.561114311  2.353528500  2.068121195  3.662022352  3.036073208
   [11]  2.973829031  1.780188322  3.946118832  1.918879151  1.824470758
   [16]  2.999991417  2.302559614  3.477477789  4.157325268  3.580404043
   [21]  3.392377138  2.363176584  3.681685209  1.065652847  2.198652744
   [26]  3.800360441  2.348160028  2.422786474  3.497869015  3.315099716
   [31]  4.054918766  3.280873060  3.186791182  2.814843655  3.761095524
   [36]  1.763220310  3.856794834  3.050387859  3.295306206  3.862437487
   [41]  3.686389685  3.195486784  3.268039227  3.033649445  3.402577639
   [46]  3.694637537  3.098803997  1.837601423  1.904559016  2.309158564
   [51]  2.546713114  3.829290867  4.040136814  3.994275331  3.875391483
   [56]  3.759994507  3.853452444  3.487160444  2.469750404  3.344584942
   [61]  3.723019361  3.253901243  3.417706013  3.420283556  3.414386988
   [66]  3.296105146  3.258196592  2.800440073  2.288883686  1.982608676
   [71]  3.480179310  2.750132322  1.951277494  2.568496466  3.831361055
   [76]  3.740110159  3.276260853  2.964428425  3.316641808  3.185754538
   [81]  3.987618685  4.082972050  3.110937119  1.293490767  0.527514279
   [86]  3.651878357  3.371524811  3.133283377  3.478158236  1.720707297
   [91]  2.087731838  2.653888226  3.038515091  2.852520227  0.592090964
   [96]  1.297412038  2.745787144  3.087521315  0.688669741  2.632688999
  [101]  0.770458400  3.124264479  2.033336163  2.299026489  0.642626941
  [106]  1.148984075  1.564483166 -0.337015033  2.456872702  1.991799593
  [111]  2.630724430  0.426842034  2.931221247  0.690171599  1.116823792
  [116]  2.886644363  0.951825798  1.072129369  2.130301237  2.232073069
  [121]  2.417049646  0.639315069  2.449440241  0.802202880  1.380573034
  [126]  3.053837776 -1.420429230  2.937889099  1.940108776  0.866741121
  [131]  2.518779278  2.456836224  2.594697237  3.670770645  3.700803280
  [136]  3.623563290  3.776622295  2.571626902  2.799937963  2.691572189
  [141]  3.745043993  2.913227797  3.801449299  3.157682896  3.735084057
  [146]  1.518264174  2.136063337  3.549924374  3.879808426  3.159516335
  [151]  3.586036444  1.964444995  3.379982948  1.033449650  3.785100698
  [156]  2.807064533  2.570823193  2.860315323  3.674057722  2.854020119
  [161]  1.164161682  3.474699974  3.852094173  3.405520439  3.793107510
  [166]  3.941418171  3.539711714  2.676273346  2.854637146  2.907936573
  [171]  3.719306231  0.912796080  2.933597326  2.405607700  3.074685812
  [176]  1.597132444  3.276872635  1.641879797  2.015620232  3.761001587
  [181]  3.400389910  2.656481743  3.029445648  3.708505630  3.692529202
  [186]  2.864082098  3.641288757  3.416702747  2.659906626  2.466082811
  [191]  0.898797154  0.873957336  3.029043674  3.482373476  3.550861120
  [196]  3.297574520  2.500671625  3.499779224  1.971122861  2.351988792
  [201]  1.538192749  2.218912840  1.423784137  2.082156181  1.273766279
  [206]  0.946896553  1.431989312  3.732721329  3.065351486  0.724882960
  [211]  2.519713879  1.781920791  2.895774364 -0.020369139  3.900358677
  [216]  1.806307435  1.917439222  0.899264336  2.996714592  2.771777391
  [221]  1.903118014  2.304287672  3.018550396  1.228020549  3.850563049
  [226]  2.882679462  3.564883947  0.778602242  0.195692956  0.521060050
  [231]  2.154811859  1.660185575  1.681107998  2.197084904  0.730890214
  [236]  3.183760166  2.302699089  0.949458778  2.349426746  3.326383591
  [241]  3.879276991  2.371184826  1.182114601  2.048728943  3.411422968
  [246]  2.499715090  3.820170879  3.771595716  3.428451061  3.657336950
  [251]  3.790361404  3.157590628  3.716688156  2.798939943  0.022324674
  [256]  2.145238400  3.182386398  3.572424412  1.851927996  2.989953041
  [261]  3.645338535  0.890769184  3.628684998  2.828729153  2.937646866
  [266]  0.061606310  2.240164995  3.258059740  3.301951885  0.897000015
  [271]  2.066240311  2.356088638  3.515242100  2.515778780  3

In [41]:
xgbclsmin

[1] 0 2 4 4 4 4 4 0 4 3 4 4 4 1 4 3 1 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4
   [37] 4 4 4 4 4 4 4 4 4 4 4 4 4 0 2 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 3 3 4 2 4 4
   [73] 3 4 4 4 4 4 4 3 4 4 4 4 0 4 4 4 4 4 4 4 4 1 4 1 4 4 0 4 0 4 0 4 0 0 4 4
  [109] 3 3 4 4 4 0 0 3 2 0 3 4 2 0 2 0 1 4 1 4 3 0 2 3 4 4 4 4 4 2 1 4 4 3 4 4
  [145] 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 2 2 4 4 4 4 4 4 4 4 4 4 0 2 4 4 0 4 1 1 4
  [181] 4 4 4 4 4 3 4 4 2 1 1 0 3 4 3 3 4 4 3 4 1 4 1 3 0 4 3 4 4 0 4 4 4 0 4 0
  [217] 4 4 3 4 0 1 4 4 4 4 3 0 0 0 3 1 4 3 0 4 2 1 4 4 4 3 0 4 4 4 4 4 4 4 4 4
  [253] 4 4 0 4 4 4 1 4 4 4 4 4 4 0 0 4 4 4 4 4 3 4 4 4 0 4 4 1 1 0 4 1 4 4 4 4
  [289] 4 4 4 4 4 4 0 4 0 4 4 3 4 4 1 4 0 4 4 0 4 4 1 1 4 0 3 1 4 0 4 4 4 2 4 4
  [325] 0 1 4 4 4 0 4 3 1 4 4 0 4 4 4 3 4 0 0 4 3 3 0 4 0 2 3 0 0 3 0 1 4 1 0 2
  [361] 4 4 4 1 2 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
  [397] 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4
  [433] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 3 4 4 4
  [469] 4 0 4 3 4 4 4 4 4 4 1 4 1 4 2 4 2 1 4 4 3 4 3 4 4 4 4 1 4 3 4 4 4 4 4 4
  [505] 4 4 4 4 4 1 4 4 0 4 4 4 3 3 3 4 4 4 4 4 0 4 4 3 4 4 4 1 4 4 4 4 1 4 4 4
  [541] 4 1 4 4 1 4 1 4 4 2 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4
  [577] 4 4 4 4 4 4 2 0 4 4 4 3 4 2 4 4 4 4 4 4 4 0 0 4 3 0 0 4 2 1 4 0 4 4 4 4
  [613] 4 4 2 0 0 4 4 0 4 4 4 4 4 4 3 4 0 2 4 1 4 4 2 4 4 4 4 4 2 4 4 4 4 1 3 4
  [649] 0 3 4 4 4 1 4 0 4 1 0 0 3 1 4 4 3 1 0 3 0 4 4 2 1 4 3 4 4 3 4 3 4 4 4 4
  [685] 3 0 4 4 0 0 0 0 0 4 0 0 4 0 4 3 4 3 4 1 1 0 0 4 4 4 4 4 4 4 3 0 4 0 0 4
  [721] 0 3 4 0 3 4 4 4 1 0 4 2 3 4 2 0 4 0 1 4 1 0 0 4 4 1 0 4 4 0 4 0 4 4 3 3
  [757] 0 1 4 4 4 4 4 3 4 3 4 4 2 4 4 0 3 4 4 4 3 2 2 4 4 4 4 0 4 0 3 0 0 4 4 4
  [793] 3 3 0 2 0 3 0 4 0 1 4 1 4 0 4 4 4 4 4 4 0 4 0 4 4 2 4 3 4 4 0 4 4 4 4 1
  [829] 4 4 3 0 4 4 4 4 4 4 0 0 4 4 4 0 0 4 0 0 0 0 4 4 4 4 3 4 4 3 4 4 4 4 4 4
  [865] 4 4 4 0 4 2 4 4 4 4 3 4 4 3 4 4 2 4 3 4 4 4 4 4 4 4 4 4 4 4 4 0 2 4 4 4
  [901] 0 0 0 1 1 0 0 1 4 3 0 2 3 1 3 0 3 0 0 2 4 0 4 0 0 0 2 4 3 1 1 0 2 4 0 4
  [937] 3 4 2 3 4 4 3 4 0 1 0 0 2 0 4 4 4 4 4 4 4 2 4 4 0 2 4 4 4 4 0 0 4 0 0 4
  [973] 0 3 4 4 2 4 4 4 0 4 1 4 4 4 4 4 4 4 0 2 4 1 3 4 0 0 0 4 0 0 1 0 1 0 3 3
 [1009] 1 0 0 3 0 0 4 0 0 4 2 3 4 4 0 0 0 0 0 2 4 4 0 1 4 4 2 0 4 4 4 0 4 0 0 4
 [1045] 4 4 4 4 4 2 4 4 0 3 4 0 4 0 0 4 0 0 0 4 4 1 0 2 4 4 4 4 4 4 4 4 4 4 4 3
 [1081] 2 4 3 2 1 1 0 2 0 1 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 1 3 4
 [1117] 4 0 4 4 4 4 0 3 4 4 4 4 4 4 4 2 1 4 2 0 4 2 0 0 1 4 4 4 4 2 4 4 4 4 4 4
 [1153] 4 4 4 4 4 4 4 0 0 1 1 4 0 2 4 4 4 4 4 4 4 4 4 4 4 4 2 1 0 1 4 1 3 1 4 4
 [1189] 4 4 4 0 4 4 4 4 4 0 0 1 4 4 4 4 3 4 4 2 4 0 0 4 4 4 3 1 4 4 4 1 0 4 2 4
 [1225] 0 4 4 4 2 4 4 4 4 4 4 1 4 4 4 4 4 4 4 1 0 4 4 4 2 4 4 4 4 0 4 4 2 4 4 3
 [1261] 4 4 4 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 0 2 4 4 4 4 4 4 1 4 4 4
 [1297] 4 4 1 4 4 2 4 4 4 4 3 3 4 4 4 4 4 4 4 4 4 4 4 1 3 4 3 4 4 4 3 4 3 3 0 4
 [1333] 4 4 4 4 3 4 4 4 3 4 3 4 4 4 4 4 4 3 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 [1369] 4 4 3 3 4 4 4 4 4 4 3 4 4 2 3 4 4 0 4 4 4 4 3 4 4 4 4 4 3 4 4 2 4 4 4 4
 [1405] 4 4 4 4 0 0 4 3 4 3 4 4 4 2 4 4 3 4 4 1 4 4 4 4 4 4 4 0 0 4 0 4 2 4 4 4
 [1441] 4 4 0 3 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 4 3 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4
 [1477] 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 3 4 4 4 4 4 3 2 2 4 4 4 4 4 4 4 4 3 4 4 4
 [1513] 4 4 4 4 4 4 2 4 4 2 4 4 4 2 0 4 4 4 4 1 4 4 4 4 3 4 4 2 2 4 4 3 4 4 1 4
 [1549] 4 4 4 4 4 4 4 4 4 0 4 4 4 4 2 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4
 [1585] 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 3 4 4 4 4 4 4 4 3 4 4 4 4 3
 [1621] 1 4 4 2 2 4 4 4 1 4 1 4 1 4 4 2 4 4 4 4 4 4 4 3 4 4 0 4 4 4 4 4 4 3 4 4
 [1657] 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 2 3 4 4 3 4 1 4 4 4 4 4 4 4 4 3 1 3 4 4 4
 [1693] 4 4 4 4 4 4 4 3 3 4 4 2 4 4 4 2 4 3 4 4 1 4 4 4 4 4 4 4 4 3 4 4 4 3 4 4
 [1729] 2 3 4 4 4 4 0 2 4 4 4 4 0 0 4 4 4 1 4 4 4 4 4 0 4 4 0 4 4 3 4 4 3 4 4 4
 [1765] 3 4 2 4 2 0 0 4 4 4 2 4 4 4 1 4 4 3 4 4 4 4 4 4 0 2 4 0 4 2 4 4 4 4 4 4
 [18

In [42]:
lgbregmin

[1]  1.251387072  2.145488862  3.036267734  3.404002531  3.711048277
    [6]  3.628803130  3.198184019  1.610292511  3.531293856  3.151621900
   [11]  3.402754280  3.529695132  3.651448875  2.108438248  2.858087232
   [16]  2.543219230  2.386265766  3.022852720  3.525817639  3.462802938
   [21]  3.475989522  3.814582051  2.300557967  1.185440312  3.549980678
   [26]  3.381716057  3.593489206  3.653799786  3.738563303  3.496471378
   [31]  3.089817917  4.014375183  3.300653520  3.723920594  3.705404762
   [36]  4.035450761  3.569394100  3.221253856  3.695369120  3.649473680
   [41]  3.665356924  4.060785925  3.823404362  3.804942067  3.376182707
   [46]  3.776549293  3.909041376  2.377406299  2.463598270  2.838445354
   [51]  2.988276612  3.705471203  3.969462088  3.724767887  3.837738067
   [56]  3.731036580  3.721089108  3.645090091  3.209027234  3.033629404
   [61]  3.690450262  3.328734125  3.822817632  3.671653894  3.288176351
   [66]  2.329194639  3.221966267  3.202214972  2.205218352  2.055557084
   [71]  3.231788132  3.479062387  1.426579519  3.624432364  3.748397814
   [76]  3.686968361  3.742479962  3.627254570  3.647048162  3.468197302
   [81]  3.650109171  3.919084842  3.816719359  2.040295416  2.662362825
   [86]  3.148139053  2.734095784  3.238365888  3.206825792  2.084470290
   [91]  3.233341455  2.340186403  3.015031868  2.674304397  1.849222844
   [96]  2.133990235  2.274082339  2.396324421  2.148142061  3.155861865
  [101]  1.701716542  3.099804532  2.185512393  1.996920514  1.738246503
  [106]  2.187681908  2.038073485  1.601194378  2.314997419  2.112810358
  [111]  2.456207390  1.618145048  2.553061478  1.829206817  1.546086139
  [116]  2.938566966  1.959072711  1.169679249  1.710834382  2.632715019
  [121]  2.190053884  0.779677694  2.419377814  1.246455226  1.692136770
  [126]  2.399794565  1.271799956  3.293724184  1.958548256  1.665681283
  [131]  2.987968077  2.807785974  3.122971626  3.707881310  3.212460825
  [136]  3.688260853  3.578173522  2.587937624  3.215658837  2.904211665
  [141]  3.527268393  2.690723517  3.761922491  3.617753113  3.612788992
  [146]  2.053253672  3.458884441  2.966909548  3.693631575  3.269646625
  [151]  3.455058482  3.709330356  3.030191983  1.492304172  3.731972685
  [156]  3.494933381  3.714224334  3.592944536  3.093058091  3.174561194
  [161]  2.047630247  3.259367937  3.789710405  2.766371401  3.609437922
  [166]  3.873419294  3.507540879  3.400326399  3.969685921  3.797956959
  [171]  3.756348247  1.458859737  3.366005932  3.539334407  3.624298193
  [176]  1.426870243  3.824441324  2.078818020  2.826197452  3.611526702
  [181]  3.248482629  2.837448748  3.783062897  3.052230565  2.869309639
  [186]  2.607565912  2.829671860  2.442118077  2.961232800  2.426365056
  [191]  1.248037053  1.052232564  2.715426712  3.560791441  3.518041070
  [196]  3.269902986  2.066947877  2.825453666  1.510914898  2.888924552
  [201]  1.653176641  2.813521139  1.745924623  2.753207423  1.542082249
  [206]  2.410618676  2.017338752  3.312990661  2.710356139  1.182991223
  [211]  1.935825797  1.804858871  2.285477288  1.410510696  2.651043604
  [216]  1.973465558  2.788021886  2.986190741  2.918930362  3.211291181
  [221]  2.798772345  2.798796293  2.393672025  1.561329533  3.332729955
  [226]  2.538117164  2.929560239  1.321901133  0.864473191  1.280083874
  [231]  2.246752886  1.071089352  2.559283246  2.277622245  2.068452343
  [236]  2.327716710  2.409578122  1.477492637  2.826983063  3.977448707
  [241]  3.667696697  2.704263249  1.732476561  3.614115672  3.277005978
  [246]  3.581826655  3.580907477  3.783307707  3.247891485  3.455974298
  [251]  3.800953391  3.994829307  3.694624377  3.045189258  1.568248689
  [256]  2.305920483  2.108985669  3.546019802  2.463315859  3.536202350
  [261]  3.802593660  1.441717859  3.300009373  3.720217717  2.559651406
  [266]  0.888835691  1.815169336  2.824934646  3.752146881  2.182234613
  [271]  1.978140234  3.023650387  3.352404499  2.434901314  3

In [43]:
lgbclsmin

0.253677708,0.13919981,0.149145306,0.16293370,0.2950435
0.201056448,0.03351485,0.463247086,0.05482935,0.2473523
0.039152709,0.05918951,0.067841484,0.16597306,0.6678432
0.046602685,0.11408003,0.073881628,0.11042980,0.6550058
0.007881825,0.01103064,0.026559817,0.06549233,0.8890354
0.032838830,0.01720774,0.012701712,0.07616760,0.8610841
0.040010330,0.05173102,0.014446379,0.09427773,0.7995345
0.407101686,0.05399038,0.078245582,0.29636126,0.1643011
0.064457255,0.03561749,0.042236628,0.10673560,0.7509530
0.146466131,0.02036969,0.017778069,0.21559725,0.5997889
0.035075793,0.28005199,0.006830478,0.04515498,0.6328868


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
head(lgbclsminr)

0
2
3
0
2
0


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
0,1.171583,0,1.251387
2,2.156448,2,2.145489
4,3.519930,3,3.036268
4,2.976367,0,3.404003
4,3.802013,2,3.711048
4,3.561114,0,3.628803


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
0,0.7868394,1,1.180393
0,1.7219460,2,1.555539
3,2.8663435,3,2.456508
1,1.7196378,0,1.922131
4,3.7095017,2,3.735557
4,4.0608888,4,3.596816


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
0,0.7868394,1,1.180393
0,1.7219460,2,1.555539
3,2.8663435,3,2.456508
1,1.7196378,0,1.922131
4,3.7095017,2,3.735557
4,4.0608888,4,3.596816


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [56]:
save(result_list, file = "yearly_thetaf_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
3.7210128,3.7210128,3.7210128
2.7225070,2.7225070,2.7225070
2.4123126,2.4123126,2.4123126
0.8011491,0.8011491,0.8011491


## importance

In [ ]:
# Gain: 这是每个特征在模型中所有树上的平均增益。增益可以理解为该特征在模型中的贡献度，增益越大，说明该特征对模型的贡献越大。
# Cover: 这是每个特征在模型中所有树上的平均覆盖度。覆盖度可以理解为该特征在模型中的使用频率，覆盖度越大，说明该特征在模型中被使用的次数越多。
# Frequency: 这是每个特征在模型中所有树上的平均频率。频率可以理解为该特征在模型中的出现频率，频率越大，说明该特征在模型中出现的次数越多。

In [57]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_cl)

# 打印特征重要性
print(importance_matrix)


        Feature       Gain      Cover  Frequency
         <char>      <num>      <num>      <num>
 1:   curvature 0.15058585 0.09117408 0.08755012
 2:   linearity 0.13516993 0.08837305 0.07668883
 3:      x_acf1 0.07034008 0.06776172 0.06466020
 4:     x_acf10 0.06775021 0.06296093 0.06400041
 5: diff2_acf10 0.05842181 0.06579993 0.07354210
 6:       trend 0.05836673 0.05859749 0.06029539
 7:  diff2_acf1 0.05660626 0.07315619 0.07227326
 8:   user_time 0.05462205 0.07085140 0.07836370
 9:     e_acf10 0.05348131 0.05721975 0.07075065
10:      e_acf1 0.05327187 0.07436363 0.06643658
11:     entropy 0.05110668 0.06445999 0.06795919
12:       spike 0.05021694 0.06611301 0.05730092
13:  diff1_acf1 0.04942916 0.06253158 0.05862051
14: diff1_acf10 0.04663801 0.05148490 0.06232553
15:      length 0.04399309 0.04515236 0.03923260


In [58]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

        Feature       Gain      Cover  Frequency
         <char>      <num>      <num>      <num>
 1:   curvature 0.18222386 0.11321666 0.08920000
 2:   linearity 0.14979830 0.11157860 0.07720000
 3:      x_acf1 0.09563202 0.06252287 0.07260000
 4:     x_acf10 0.07359215 0.08551806 0.07446667
 5:      length 0.06212269 0.07436781 0.06033333
 6:       trend 0.05421155 0.05518243 0.05826667
 7:       spike 0.04759240 0.05680579 0.05906667
 8:      e_acf1 0.04597609 0.07671485 0.06506667
 9:  diff2_acf1 0.04491792 0.07201584 0.06806667
10: diff2_acf10 0.04410917 0.05190645 0.06640000
11:     e_acf10 0.04338906 0.05046592 0.06746667
12:     entropy 0.04070244 0.04468504 0.06066667
13:  diff1_acf1 0.04066085 0.04230515 0.05800000
14:   user_time 0.03766223 0.04067801 0.06373333
15: diff1_acf10 0.03740925 0.06203653 0.05946667


In [59]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'y_thetaf_imp_mean.csv')

In [62]:
importance

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.24040799,0.12015925,0.09618047
diff1_acf1,0.03357167,0.04425215,0.05168352
diff1_acf10,0.03703120,0.05428803,0.06098632
diff2_acf1,0.03820004,0.06744448,0.06092500
diff2_acf10,0.03803781,0.06299547,0.06242190
e_acf1,0.04127852,0.07345144,0.06687524
e_acf10,0.03804131,0.06113255,0.06593844
entropy,0.03659988,0.05099637,0.05753308
length,0.05099953,0.05766005,0.05504987


In [37]:
# 使用saveRDS保存模型到.rds文件
saveRDS(xgb_mean_reg, file = "Y_thetafxgbreg.rds")
saveRDS(xgb_mean_cl, file = "Y_thetafxgbcls.rds")
saveRDS(lgb_mean_reg, file = "Y_thetaflgbreg.rds")
saveRDS(lgb_mean_cl, file = "Y_thetaflgbcls.rds")